# Imports

In [31]:
from typing import List
import json 
import re
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
from pymystem3 import Mystem

[nltk_data] Downloading package stopwords to /home/diveev/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data loading

In [2]:
data = pd.read_csv("../data/kp_films.csv")
data = data.dropna().reset_index().drop(columns='index')

In [3]:
data

,kp_id,rating_kp,genres,description
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,..."
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем..."
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди..."
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд..."
...,...,...,...,...
18494,806159,5.31,['семейный' 'аниме' 'приключение' 'фантастика'...,"Сатоши готовится к новому турниру, где парню н..."
18495,625381,6.40,['русский' 'мультфильм' 'для' 'ребенок' 'детск...,На нелепом и забавном мужчине по прозвищу Пара...
18496,4535755,-1.00,['семейный' 'мультфильм' 'иностранный' 'для' '...,Очаровательная сказка о маленьком лягушонке с ...
18497,1111361,6.60,['драма'],Криминальная драма о реальной истории жизни си...


# Data preprocessing

In [4]:
genres = data.genres.to_list()

for i in range(len(genres)):
    genres[i] = genres[i].replace('[', '')
    genres[i] = genres[i].replace(']', '')
    genres[i] = genres[i].replace(',', '')
    genres[i] = genres[i].replace("'", '')
    genres[i] = genres[i].split()

In [5]:
genres_corpus = []

for genres_list in genres:
    genres_corpus += genres_list

genres_corpus = set(genres_corpus)

In [6]:
genres_corpus = genres_corpus.difference(stopwords.words("russian")) #removing stopwords from genres

In [7]:
for genre in genres_corpus:
    data[genre] = [1 if genre in genres[i] else 0 for i in range(data.shape[0])]

In [8]:
data

,kp_id,rating_kp,genres,description,романтика,аниме,ссср,шоу,сказка,мировой,...,триллер,спорт,ребенок,военный,мистика,детектив,вокруг,полнометражный,комедия,психология
0,12819,7.70,['триллер' 'мелодрама' 'криминал'],"Миллиардер Томас Краун, пресыщенный финансист,...",0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1260096,-1.00,['семейный' 'для' 'кино' 'ребенок'],Банда саранчи во главе с безжалостным Боссом о...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,894673,4.40,['вестерн' 'иностранный' 'триллер' 'мистика' '...,"В горах живёт обособленная община, возглавляем...",0,0,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0
3,485364,6.06,['семейный' 'для' 'кино' 'короткометражный' 'р...,"Фильм высмеивает тех, кто понимает трудовую ди...",0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,1289632,6.40,['семейный' 'приключение' 'кино'],"В скучном городке Линдсборо всё меняется, когд...",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18494,806159,5.31,['семейный' 'аниме' 'приключение' 'фантастика'...,"Сатоши готовится к новому турниру, где парню н...",0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18495,625381,6.40,['русский' 'мультфильм' 'для' 'ребенок' 'детск...,На нелепом и забавном мужчине по прозвищу Пара...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
18496,4535755,-1.00,['семейный' 'мультфильм' 'иностранный' 'для' '...,Очаровательная сказка о маленьком лягушонке с ...,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
18497,1111361,6.60,['драма'],Криминальная драма о реальной истории жизни си...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
genre_to_num = {genre : (data[genre] != 0).sum() for genre in genres_corpus if (data[genre] != 0).sum() > 50}

In [10]:
#Drop the least appearing genres

preprocessed_data = data[list(genre_to_num.keys())]
descriptions = data[~(preprocessed_data==0).all(axis=1)].description
preprocessed_data = preprocessed_data.loc[~(preprocessed_data==0).all(axis=1)].reset_index().drop(columns='index')
preprocessed_data['description'] = descriptions

In [11]:
preprocessed_data.head(5)

,аниме,ссср,шоу,сказка,мировой,мультфильм,семейный,тв,драма,хотеть,...,криминал,триллер,спорт,ребенок,военный,мистика,детектив,полнометражный,комедия,description
0,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,"Миллиардер Томас Краун, пресыщенный финансист,..."
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,Банда саранчи во главе с безжалостным Боссом о...
2,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,"В горах живёт обособленная община, возглавляем..."
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,0,"Фильм высмеивает тех, кто понимает трудовую ди..."
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"В скучном городке Линдсборо всё меняется, когд..."


In [12]:
descriptions = preprocessed_data.description

In [13]:
def normalize_text(strings:List[str]) -> List[str]:
    normalized = []
    
    for string in strings:
        string = re.findall(r"\w+", str(string))
        string = " ".join(string)
        string = re.sub(r"\s+", " ", string)
        string = string.lower()
        
        normalized.append(string)
        
    return normalized

In [14]:
def filter_stopwords(strings:List[str]) -> List[str]:
    filtered = []
    
    for string in strings:
        words = string.split()
        words = [w for w in words if w not in stopwords.words("russian")]
        string = " ".join(words)
        
        filtered.append(string)
        
    return filtered

In [15]:
def lemmatize_words(strings:List[str]) -> List[str]:
    lemmatized = []
    stem = Mystem()
    
    for string in strings:
        lemmas = stem.lemmatize(string)
        lemmas[-1] = lemmas[-1].replace("\n", "")
        string = ''.join(lemmas)
        
        lemmatized.append(string)
        
    return lemmatized

In [16]:
def preprocessing_pipeline(strings:List[str]) -> List[str]:
    strings = normalize_text(strings)
    strings = filter_stopwords(strings)
    strings = lemmatize_words(strings)
    
    return strings

In [17]:
corpus = preprocessing_pipeline(descriptions)

In [20]:
id_to_text = {i : corpus[i] for i in range(len(corpus))}

In [32]:
with open("../data/movie_descriptions.json", "w") as f:
    f.write(json.dumps(id_to_text))

In [38]:
with open("../data/movie_descriptions.json") as f:
    data = json.loads(f.read())
descriptions = list(data.values())

In [40]:
vectorizer = TfidfVectorizer()
corpus = vectorizer.fit_transform(descriptions)

In [41]:
corpus

<18496x51504 sparse matrix of type '<class 'numpy.float64'>'
	with 1061098 stored elements in Compressed Sparse Row format>